In [1]:
import numpy as np
import mpi4py.rc
mpi4py.rc.initialize = False  
from mpi4py import MPI
MPI.Init()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nr_proc = comm.Get_size()

In [66]:
#Know where I am:

y_pos = rank%np.sqrt(nr_proc)
x_pos = (rank-rank%np.sqrt(nr_proc))/np.sqrt(nr_proc)

ID = np.array([x_pos,y_pos])

x_range = np.array(ID[1]/np.sqrt(nr_proc),)

In [95]:
DIM = 2
sigma = 0.05
nx = 10
N = nx**2
d_perfect = 2**(1/6)*sigma
epsilon = 10
BoxSize = 1
Rcutoff = 2.5*sigma

x_range = np.array([ID[1]/np.sqrt(nr_proc),(ID[1]+1)/np.sqrt(nr_proc)])
y_range = np.array([ID[0]/np.sqrt(nr_proc),(ID[0]+1)/np.sqrt(nr_proc)])



core_dist = BoxSize/np.sqrt(nr_proc)
dist_init = np.linspace(0,0.8,nx)
upper_x = x_range[1]
lower_x = x_range[0]
x_init = dist_init[np.where(upper_x >= dist_init)]
x_init = x_init[np.where(lower_x < x_init)]

upper_y = y_range[1]
lower_y = y_range[0]
y_init = dist_init[np.where(upper_y >= dist_init)]
y_init = y_init[np.where(lower_y < y_init)]
#core_dist = BoxSize/np.sqrt(nr_proc)
nx, ny = len(x_init),len(y_init)

pos = np.zeros([nx*ny,DIM])
for i in range(nx):
    for j in range(ny):
        pos[i+nx*j] = [x_init[i]*BoxSize,y_init[j]*BoxSize]
        

        
print("This is process {} and the initial posistions I have are {}.".format(rank, pos))

This is process 7 and the initial posistions I have are [[0.35555556 0.71111111]
 [0.44444444 0.71111111]
 [0.53333333 0.71111111]
 [0.62222222 0.71111111]
 [0.35555556 0.8       ]
 [0.44444444 0.8       ]
 [0.53333333 0.8       ]
 [0.62222222 0.8       ]].


In [ ]:
def Compute_Forces(pos,epsilon,BoxSize,DIM,N):
    Sij = np.zeros(DIM)
    Rij = np.zeros(DIM)
    acc = np.zeros([N,DIM])
    for i in range(N-1):
        for j in range(i+1,N):
            Sij = (pos[i,:]-pos[j,:])
            
            for l in range(DIM):
                if (np.abs(Sij[l])>0.5):
                    Sij[l] = Sij[l]-np.copysign(1,Sij[l])
            
            Rij = BoxSize*Sij
            
            Rsqij = np.dot(Rij,Rij)
            
            if (Rsqij<Rcutoff**2):
                r2 = 1/Rsqij
                r6 = r2**2
                r12 = r6**3
                dphi = epsilon*24*r2*(2*sigma**12*r12-sigma**6*r6)
                
                acc[i,:] = acc[i,:]+dphi*Sij
                acc[j,:] = acc[j,:]-dphi*Sij#
    return acc                

In [105]:
def rank_from_ID(ID):
    return int(3*ID[0]+ID[1])

In [121]:
def send_positions_near_sides(pos,ID,x_range,y_range,Rcutoff):
    upper_x = x_range[1]
    lower_x = x_range[0]
    upper_y = y_range[1]
    lower_y = y_range[0]
    #Left wall
    LEFT_WALL = pos[np.where(pos[:,0]<= lower_x + Rcutoff)]
    comm.send(LEFT_WALL,dest=rank_from_ID((ID[0],(ID[1]-1)%3)),tag="FROM RIGHT")
    
    #Right wall
    RIGHT_WALL = pos[np.where(pos[:,0]>= upper_x - Rcutoff)]
    comm.send(RIGHT_WALL,dest=rank_from_ID((ID[0],(ID[1]+1)%3)),tag="FROM LEFT"
              
    #Lower wall
    LOWER_WALL = pos[np.where(pos[:,1]>= lower_y + Rcutoff)]
    comm.send(LOWER_WALL,dest=rank_from_ID(((ID[0]-1)%3,ID[1])),tag="FROM ABOVE")
              
    #Upper wall
    UPPER_WALL = pos[np.where(pos[:,1]<= upper_y - Rcutoff)]
    comm.send(UPPER_WALL,dest=rank_from_ID(((ID[0]+1)%3,ID[1])),tag="FROM BELOW")


In [115]:
LEFT_WALL = pos[np.where(pos[:,0]<= lower_x + Rcutoff)]

In [124]:
pos

array([[0.35555556, 0.71111111],
       [0.44444444, 0.71111111],
       [0.53333333, 0.71111111],
       [0.62222222, 0.71111111],
       [0.35555556, 0.8       ],
       [0.44444444, 0.8       ],
       [0.53333333, 0.8       ],
       [0.62222222, 0.8       ]])

In [119]:
for i in LEFT_WALL:
    print(i)

[0.35555556 0.71111111]
[0.44444444 0.71111111]
[0.35555556 0.8       ]
[0.44444444 0.8       ]


In [117]:
lower_x + Rcutoff

0.4583333333333333

In [113]:
pos[:,0]

array([0.35555556, 0.44444444, 0.53333333, 0.62222222, 0.35555556,
       0.44444444, 0.53333333, 0.62222222])

In [112]:
pos

array([[0.35555556, 0.71111111],
       [0.44444444, 0.71111111],
       [0.53333333, 0.71111111],
       [0.62222222, 0.71111111],
       [0.35555556, 0.8       ],
       [0.44444444, 0.8       ],
       [0.53333333, 0.8       ],
       [0.62222222, 0.8       ]])

In [118]:
ID

array([2., 1.])